# STUDENT INTERVENTION SYSTEM

- **In the classroom, interventions are activities that you would use to help students become successful in their classwork or decrease negative behavior towards others.**


- **The greatest benefit of an the intervention approach is that it eliminates a “wait to fail” situation because students get help promptly within the general education setting. As soon as assessment data indicates a problem area for a student or a group of students, interventions are put into place to address these concerns.**


- **It's important for schools to focus on intensive interventions because it gives an opportunity for the schools to figure out ways to serve their neediest students.**

**QUESTION 1**

Our goal for the project is to identify those students who might need an early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression and whydo we say that?

**ANSWER:**

Our problem comes under the Classification category because our problem statement is whether the student will pass or not? and the target variable is 'passed', which is of categorical type. It has two possible values. 'YES' or 'NO'

In [3]:
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

data = pd.read_csv(r"C:\Users\Nandhitha\Desktop\Datasets\student data.csv")
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes


## DATA EXPLORATION

**QUESTIONS TO BEGIN WITH :**

* What total number of students in the class?


* How many characteristics is defined for each student?


* Can you find the number of students who passed and number of students who failed?


* What is the graduation rate of the class?

In [5]:
total_students = data.shape[0]
total_students

395

In [6]:
characteristics = data.shape[1]
characteristics

31

In [11]:
(data['passed']=='yes').value_counts()

True     265
False    130
Name: passed, dtype: int64

In [13]:
pass_count = data['passed'].value_counts()[0]
pass_count

265

In [14]:
fail_count = data['passed'].value_counts()[1]
fail_count

130

In [16]:
graduation_rate = (pass_count)/total_students * 100
graduation_rate

67.08860759493672

**ANSWERS:**
* Total number of students: 395


* Number of characteristics: 31


* Number of students who passed: 265


* Number of students who failed: 130


* Graduation rate of the class: 67.08

## DATA PREPROCESSING
Tasks to be done to prepare the data initially:
* Extract the characteristics columns individually


* Extract our target column 'passed'


* Separate the data into characteristic data and target data

In [18]:
char_columns = list(data.columns[:-1])
target_column = data.columns[-1] 

In [24]:
char = data[char_columns]
char

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,yes,no,no,4,3,4,1,1,3,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,yes,no,5,3,3,1,1,3,4
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,yes,no,4,3,2,2,3,3,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,yes,3,2,2,1,1,5,2
4,GP,F,16,U,GT3,T,3,3,other,other,...,yes,no,no,4,3,2,1,2,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,yes,no,no,5,5,4,4,5,4,11
391,MS,M,17,U,LE3,T,3,1,services,services,...,yes,yes,no,2,4,5,3,4,2,3
392,MS,M,21,R,GT3,T,1,1,other,other,...,yes,no,no,5,5,3,3,3,3,3
393,MS,M,18,R,LE3,T,3,2,services,other,...,yes,yes,no,4,4,1,3,4,5,0


In [25]:
target = data[target_column]
pd.DataFrame(target)

,passed
0,no
1,no
2,yes
3,yes
4,yes
...,...
390,no
391,yes
392,no
393,yes


**PREPROCESSING COLUMNS**

- In our data, there are several non-numeric columns that need to be converted. Many of them are (yes/no) values. These values can be reasonably converted into (1/0) binary values, so that it can be conviniently fed to fit the model.


- For the other columns that consist of categorical variables, we can seperate and create as many columns with possible correct values and assign '1' to the suitable variables and '0' to all others by using a function to perform this transformation.

In [28]:
def preprocess(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    output = pd.DataFrame(index = X.index)
    for col, col_data in X.iteritems():
        
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)  
        output = output.join(col_data)
    
    return output

char = preprocess(char)
char

,school_GP,school_MS,sex_F,sex_M,age,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,1,0,1,0,18,0,1,1,0,1,...,1,0,0,4,3,4,1,1,3,6
1,1,0,1,0,17,0,1,1,0,0,...,1,1,0,5,3,3,1,1,3,4
2,1,0,1,0,15,0,1,0,1,0,...,1,1,0,4,3,2,2,3,3,10
3,1,0,1,0,15,0,1,1,0,0,...,1,1,1,3,2,2,1,1,5,2
4,1,0,1,0,16,0,1,1,0,0,...,1,0,0,4,3,2,1,2,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,0,1,0,1,20,0,1,0,1,1,...,1,0,0,5,5,4,4,5,4,11
391,0,1,0,1,17,0,1,0,1,0,...,1,1,0,2,4,5,3,4,2,3
392,0,1,0,1,21,1,0,1,0,0,...,1,0,0,5,5,3,3,3,3,3
393,0,1,0,1,18,1,0,0,1,0,...,1,1,0,4,4,1,3,4,5,0


**DATA SPLIT : TRAINING AND TESTING**

We have converted all categorical characteristics into numericals. Next, we will try to split the data (both characteristics and its corresponding labels) into train and test sets.


* Randomly shuffle and split the data (char, target) into training and testing sets.


* Fix 300 training points (nearly 75%) and 95 testing points (nearly 25%).


* Store the split results in X_train, X_test, y_train, and y_test.

In [32]:
train = 300
test = char.shape[0] - train

X_train = char.head(train)
X_test = char.tail(test)
y_train = target.head(train)
y_test = target.tail(test)

In [38]:
X_train.shape, y_train.shape

((300, 48), (300,))

In [39]:
X_test.shape, y_test.shape

((95, 48), (95,))

**TRAINING AND EVALUATION OF MODELS**

We will first choose 3 supervised learning models that are appropriate for our problem. We will then try to fit the model using varying sizes of training data (eg: 100 data points, 200 data points, and 300 data points) and measure the F1 score. We will also calculate the training time, prediction time on the training set.

We will choose the following models from scikit-learn:

* Random Forest


* Logistic Regression


* Adaboost

**QUESTION 2**

Why have we chosen these 3 models? What can we say about the strengths and weakness of each of the models?

**ANSWER:**

**1. Random Forest** :
 - This classifier is used in fileds like credit scoring, crime risk, medical diagnosis, failure prediction etc.
 
 
 - Not prone to overfit and not computationally expensive which is what we want, to take less time.
 
 
 - They’re not easily interpretable.
 
 
 - Random forest are not prone to overfit as we have lots features containing integers, categorical and binary. This eliminates the risk of overfitting and not computationally expensive as advised by supervisors.
 
 
**2. Logistic Regression**
 - Used in Financial forecasting, Software cost prediction, software effort prediction and software quality assurance, etc
 
 
 - It doesn't assume a linear relationship between the features and target variable. (Good for us as we might have with lots of features)
 
 
 - It requires large sample size in order to perform well. (Sad we don't have it but let's try.
 
**3. AdaBoost**
 - This classifier is used in fileds biology, speech processing, computer vision etc.
 
 
 - Less tuning of parameters required and computational friendly.
 
 
 - Sensitive to noisy data and outliers.

**SETTING UP THE MODELS**

We will first initialize three helper functions which can be used for training and testing the three supervised learning models. The description of the functions are as follows:

- **classifier_train** - It takes as input a classifier and training data and fits the classifier to the data.


- **predict_label** - It takes as input a fit classifier, its features, and a target labeling to makes predictions using the F1 score.


- **predict_train** - It takes as input a classifier, the training and testing data and will report the F1 score for training and testing data separately.

In [41]:
def classifier_train(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
def predict_label(clf, characteristics, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''

    start = time()
    y_pred = clf.predict(characteristics)
    end = time()
    return f1_score(target.values, y_pred, pos_label='yes')


def predict_train(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    classifier_train(clf, X_train, y_train)

**PERFORMANCE METRICS**

We will now import the three supervised learning models and run the predict_train function for each of the models.

Tasks that we have to do:

- Import the three supervised learning models.


- Initialize the models and store them in clf_A, clf_B and clf_C.


- Create different training set sizes to be used to train each model without re-shuffling.


- Try to fit each model with each training set size and make predictions on the test set.  

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

clf_A = LogisticRegression(random_state=53)
clf_B = RandomForestClassifier(random_state=53)
clf_C = AdaBoostClassifier(random_state=53)

X_train_100 = X_train.head(100)
y_train_100 = y_train.head(100)

X_train_200 = X_train.head(200)
y_train_200 = y_train.head(200)

X_train_300 = X_train.head(300)
y_train_300 = y_train.head(300)

**RANDOM FOREST**

In [44]:
predict_train(clf_B, X_train_100, y_train_100, X_test, y_test);
    
predict_train(clf_B, X_train_200, y_train_200, X_test, y_test);
    
predict_train(clf_B, X_train_300, y_train_300, X_test, y_test);

**LOGISTIC REGRESSION**

In [49]:
predict_train(clf_A, X_train_100, y_train_100, X_test, y_test);
    
predict_train(clf_A, X_train_200, y_train_200, X_test, y_test);
    
predict_train(clf_A, X_train_300, y_train_300, X_test, y_test);

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


**ADA BOOST**

In [51]:
predict_train(clf_C, X_train_100, y_train_100, X_test, y_test);
    
predict_train(clf_C, X_train_200, y_train_200, X_test, y_test);
    
predict_train(clf_C, X_train_300, y_train_300, X_test, y_test);

## RESULTS

**Logistic Regression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.1940          |        0.0150          |     0.9200       |      0.6950     |
| 200               |        0.0160           |          0.0           |     0.8414       |        0.7154   |
| 300               |           0.0           |          0.0           |        0.8486    |    0.6552       |

**Random Forest**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0470          |        0.0             |      1.0         |       0.6555    |
| 200               |         0.0310          |        0.0             |      1.0         |       0.6614    |
| 300               |         0.0310          |        0.0             |      0.9975      |     0.6379      |

**AdaBoost**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.1440             |          0.0           |     0.9867       |      0.5333     |
| 200               |      0.1250             |          0.0           |     0.8920       |      0.6167     |
| 300               |     0.1710              |          0.0           |     0.8585       |     0.6726      |

## SELECTION OF BEST MODEL
We will choose from the three supervised learning models the best model to use on the student data. Then we will perform grid search optimization for the model over the entire training set (X_train and y_train) by tuning at least one of the parameter to improve upon the untuned model's F1 score. 

**QUESTION 3**

Based on the experiments performed earlier and the results, which model are we going to chose as the best model? Why? Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**ANSWER:** 
- In terms of computational cost, all model are noticed to perform well in terms of training time and predicting time. It was almost 0 for all run except for first test of 100 samples in Logistic Regression.


- Logistic Regression is the model that we need to choose. Because it took almost no time to train and predict 300 samples whereas Random Forest and AdaBoost took 1/10<sup>th</sup> of a second to train. Also the F1 score on train and the test are  good enough compared to other models. We also have to note that Logistic Regression Model gets the highest F1 score on test for 200 sample size.


- Therefore we choose 'Logistic Regression Model', also keeping in mind computational cost, resource available and performance.

## MODEL TUNING
Now we will fine tune our model using grid search, considering at least one important parameter tuned with at least 3 different values. 

Tasks to be done:

- Create a dictionary of parameters to tune for the model.


- Initialize the classifier and store it in clf.


- Create the F1 scoring function.


- Perform grid search on the classifier.


- Fit the grid search object to the training data (X_train, y_train).

In [64]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

parameters = {'C' : [0.01, 0.1, 0.5]}

clf = LogisticRegression()
f1_scorer = make_scorer(f1_score, pos_label='yes')
grid = GridSearchCV(estimator = clf, param_grid = parameters, scoring = f1_scorer)
grid= grid.fit(X_train, y_train)
clf = grid.best_estimator_

train=(predict_label(clf, X_train, y_train));
test=(predict_label(clf, X_test, y_test));

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to

In [60]:
train, test

(0.8200836820083682, 0.7894736842105263)

In [62]:
grid.best_estimator_

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

**QUESTION 5**

What is the model's final F1 score for train and test? How does that score compare to the untuned model's score?

**ANSWER:**
- In our tuned model, the time for testing and predicting is almost 0 which is what we want and it is as same for untuned models also.


- When it comes to F1 score on training, with our tuned model we are able to loose score by ~2% compared to both untuned models. When it comes to F1 score on testing, with our tuned model we are able to gain score by ~14% compared to untuned models.

## CONCLUSION

Given a less amount of data, we have implemented the Logistic Regression Model to figure out which student needs intervention to help the school achieve its target of 95% graduation rate!